In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

### replace()函数的用法
- 参数说明
    - to_replace 原值
    - value 新值
- 替换的方式
    - 单值替换
    - 多值替换
        - list
        - dict
- 替换的数据对象
    - Series
        - 额外使用 method 填充方法{pad, ffill, bfill}
        - limit 限制填充的次数
        - 组合使用： to_replace+ method+ limit 
    - DataFrame
        - to_replace 字典dict表示为某一特征下的所有的原值
        - value 新值

In [7]:
data = np.random.randint(150, size=(5, 3))
index = list('ABCDE')
columns = ('数学', '英语', '计算机')

df = DataFrame(data, index, columns)
df

,数学,英语,计算机
A,85,73,56
B,61,131,128
C,146,125,138
D,46,90,39
E,144,62,35


In [8]:
c = df['计算机']
c

A     56
B    128
C    138
D     39
E     35
Name: 计算机, dtype: int64

In [9]:
# 将计算机的不及格的成绩改为60
c.replace([39, 35], 60, inplace=True)
df

,数学,英语,计算机
A,85,73,56
B,61,131,128
C,146,125,138
D,46,90,60
E,144,62,60


In [12]:
c['A']=60
df

,数学,英语,计算机
A,85,73,60
B,61,131,128
C,146,125,138
D,46,90,60
E,144,62,60


In [15]:
# 将计算机成绩为60分的第一位同学的成绩改为前一位同学的计算机成绩
c.replace(60, method='ffill', limit=1)

A     60
B    128
C    138
D    138
E     60
Name: 计算机, dtype: int64

In [17]:
df.iloc[-1, 1] = 61
df

,数学,英语,计算机
A,85,73,60
B,61,131,128
C,146,125,138
D,46,90,60
E,144,61,60


In [19]:
# 修改英语中61分的，全部改为80分
# df.replace(61, 80)
df.replace({'英语':61 }, 80)

,数学,英语,计算机
A,85,73,60
B,61,131,128
C,146,125,138
D,46,90,60
E,144,80,60


### map() 函数的用法
- 将某一列或多列的数据映射成另一个结果（有意义的数据）
- 两种写法
    - 字典方式
        - 要求：选择的列的值作为key全部存在，如果未指定列值key，结果则为Nan。 
        - 建议：单列
    - 函数方式
        - 将选择的列的每行的值传入到指定函数中，并返回处理结果
        - 不能使用聚合函数和for循环
        - <font color=red>【注】不能对DataFrame进行map,只能对某一行或列map, 即是Series进行map映射操作的。</font>

In [27]:
# 查找数学为146 和144的学生，将其的"名次"定为"第一"和'第二'
df['数学名次'] = df['数学'].map({146:'第一名', 144:'第二名'})
df

,数学,英语,计算机,数学名次
A,85,73,60,NaN
B,61,131,128,NaN
C,146,125,138,第一名
D,46,90,60,NaN
E,144,61,60,第二名


In [29]:
# 增加"总成绩"列，计算每位同学的总成绩
df['总成绩'] = df.sum(axis=1)
df

In [39]:
def level_name(item):
    # item 是某一列（总成绩）的数值
    if item>= 400:
        return '优'
    elif item >= 300:
        return '良'
    elif item >= 270:
        return '及格'
    
    return '补考'

In [36]:
# df.drop('总成绩',axis=1, inplace=True)
df['数英平均'] = df[['数学', '英语']].mean(axis=1)

In [38]:
# 根据总成绩，标出成绩等级('优', '良', '及格', '补考')
# 400+ 优
# 300+ 良
# 270+ 及格
# 270- 补考
df['总成绩'] = df.iloc[:, :3].sum(axis=1)
df

,数学,英语,计算机,数学名次,数英平均,总成绩
A,85,73,60,NaN,79.0,218
B,61,131,128,NaN,96.0,320
C,146,125,138,第一名,135.5,409
D,46,90,60,NaN,68.0,196
E,144,61,60,第二名,102.5,265


In [40]:
# 400+ 优
# 300+ 良
# 270+ 及格
# 270- 补考
df['等级'] = df['总成绩'].map(level_name)
df

,数学,英语,计算机,数学名次,数英平均,总成绩,等级
A,85,73,60,NaN,79.0,218,补考
B,61,131,128,NaN,96.0,320,良
C,146,125,138,第一名,135.5,409,优
D,46,90,60,NaN,68.0,196,补考
E,144,61,60,第二名,102.5,265,补考


### transform() 函数
- 同map()相同, 但是不能使用dict字典

In [41]:
df['等级2'] = df['总成绩'].transform(level_name)
df

,数学,英语,计算机,数学名次,数英平均,总成绩,等级,等级2
A,85,73,60,NaN,79.0,218,补考,补考
B,61,131,128,NaN,96.0,320,良,良
C,146,125,138,第一名,135.5,409,优,优
D,46,90,60,NaN,68.0,196,补考,补考
E,144,61,60,第二名,102.5,265,补考,补考


练习：
新增两列，分别为张三、李四的成绩状态，如果分数低于90，则为"failed"，如果分数高于120，则为"excellent"，其他则为"pass"
【提示】使用函数作为map的参数

In [46]:
d = df['总成绩']

In [82]:
ds = d.sort_values(ascending=False)
ds

C    409
B    320
E    265
A    218
D    196
Name: 总成绩, dtype: int64

In [80]:
ds[ds == 196].index  # D 196 -> Series -> Index
ds.index.get_loc(ds[ds == 196].index.values[0])

0

In [92]:
ds[ds==196].index.values[0]

'D'

In [93]:
dd = ds.copy()
dd

C    409
B    320
E    265
A    218
D    196
Name: 总成绩, dtype: int64

In [104]:
def l_name(item):  # 根据总成绩，返回成绩名次
    # item是总成绩的数值
    # ds[ds == item]   -> D 196 -> Series
    # 获取item值对应的索引名
    # values -> ndarray
    index_name = dd[dd==item].index.values[0]
    
    # 获取索引名，从已排序的Series读取索引的位置
    index_value = dd.index.get_loc(index_name)
    index_value += 1
    return f'第{index_value}名'

In [105]:
df['名次'] = df['总成绩'].transform(l_name)
df

,数学,英语,计算机,数学名次,数英平均,总成绩,等级,等级2,名次
A,85,73,60,NaN,79.0,218,补考,补考,第4名
B,61,131,128,NaN,96.0,320,良,良,第2名
C,146,125,138,第一名,135.5,409,优,优,第1名
D,46,90,60,NaN,68.0,196,补考,补考,第5名
E,144,61,60,第二名,102.5,265,补考,补考,第3名


### rename() 重命名行或列标签
- index 行
- columns 列
- level 多层索引或列标签， 可以指定哪一级别，默认为一级

In [107]:
df.drop('等级2', axis=1, inplace=True)
df

,数学,英语,计算机,数学名次,数英平均,总成绩,等级,名次
A,85,73,60,NaN,79.0,218,补考,第4名
B,61,131,128,NaN,96.0,320,良,第2名
C,146,125,138,第一名,135.5,409,优,第1名
D,46,90,60,NaN,68.0,196,补考,第5名
E,144,61,60,第二名,102.5,265,补考,第3名


In [111]:
# 将 A~ E 重新命名为 具体人名
df.rename(index={'A': '李大哥',
                 'B':'王大哥',
                 'C':'狄大叔',
                 'D': '黄小妹',
                 'E': '刘小庆'}, inplace=True)

In [113]:
# 同时修改index和列
df.rename(index={'狄大叔':'狄哥'},
         columns={'等级': '是否补考'}, inplace=True)
df

,数学,英语,计算机,数学名次,数英平均,总成绩,是否补考,名次
李大哥,85,73,60,NaN,79.0,218,补考,第4名
王大哥,61,131,128,NaN,96.0,320,良,第2名
狄哥,146,125,138,第一名,135.5,409,优,第1名
黄小妹,46,90,60,NaN,68.0,196,补考,第5名
刘小庆,144,61,60,第二名,102.5,265,补考,第3名


In [114]:
data = np.random.randint(150, size=(5, 3))
index = list('ABCDE')
columns = ('数学', '英语', '计算机')

df1 = DataFrame(data, index, columns)
df2 = DataFrame(np.random.randint(150, size=(5, 3)),
                index, columns)
# 连接两个DF， 按行，并指定第一张表是2017年, 第二张表是2018年
df3 = pd.concat((df1, df2), axis=0,
          keys=('2017年', '2018年'))
df3

数学   英语  计算机
2017年 A   34   79   35
      B  119   11  120
      C   44   81   36
      D   46   97   98
      E   47  109   35
2018年 A  123  109   85
      B  105  122   61
      C   79   46   21
      D  129   49   87
      E   21   78  109

In [134]:
# 修改 2018年D的姓名为 '张三丰' ？？
# 统一针对一级或二级索引或列标签修改
# df3.rename(index={1: '张三丰', 'E': '李小龙'},
#            level=1)

df4 = df3.loc['2018年']
df4.rename(index={'D': '张三丰'}, inplace=True)

In [159]:
# 定义rename函数, 针对2018年的D行索引进行修改
last_level = None
def d_rename(item):
    global last_level
    if item.endswith('年') and last_level != item:
        last_level = item
    else:
        if last_level == '2018年' and item == 'D':
            return '张三丰'
    return item

In [160]:
df3.rename(index=d_rename)

数学   英语  计算机
2017年 A     34   79   35
      B    119   11  120
      C     44   81   36
      D     46   97   98
      E     47  109   35
2018年 A    123  109   85
      B    105  122   61
      C     79   46   21
      张三丰  129   49   87
      E     21   78  109

### 异常值检测和过滤
- describe()  查看每一列数值的统计情况
- std()

In [257]:
df1

,数学,英语,计算机
A,34,79,35
B,119,11,120
C,44,81,36
D,46,97,98
E,47,109,35


In [256]:
df1.describe()

,数学,英语,计算机
count,5.000000,5.000000,5.000000
mean,58.000000,75.400000,64.800000
std,34.489129,38.036824,41.093795
min,34.000000,11.000000,35.000000
25%,44.000000,79.000000,35.000000
50%,46.000000,81.000000,36.000000
75%,47.000000,97.000000,98.000000
max,119.000000,109.000000,120.000000


In [268]:
# df
# percentiles 查看百分比的比例统计
# include 指定统计哪些数据类型
# exclude 不统计哪些数据类型
df.describe(percentiles=[.45, .8],
            exclude=[np.float, np.object])

,数学,英语,计算机,总成绩
count,5.000000,5.000000,5.000000,5.000000
mean,96.400000,96.000000,89.200000,281.600000
std,46.500538,31.048349,40.139756,85.663878
min,46.000000,61.000000,60.000000,196.000000
45%,80.200000,86.600000,60.000000,255.600000
50%,85.000000,90.000000,60.000000,265.000000
80%,144.400000,126.200000,130.000000,337.800000
max,146.000000,131.000000,138.000000,409.000000


In [273]:
df.std(axis=1)

李大哥     64.949981
王大哥    100.676214
狄哥     122.263854
黄小妹     60.282667
刘小庆     84.818925
dtype: float64

In [275]:
df['数学'].replace(61, np.nan, inplace=True)
df

,数学,英语,计算机,数学名次,数英平均,总成绩,是否补考,名次
李大哥,85.0,73,60,NaN,79.0,218,补考,第4名
王大哥,NaN,131,128,NaN,96.0,320,良,第2名
狄哥,146.0,125,138,第一名,135.5,409,优,第1名
黄小妹,46.0,90,60,NaN,68.0,196,补考,第5名
刘小庆,144.0,61,60,第二名,102.5,265,补考,第3名


In [280]:
df.std(skipna=False) # skipna=True 跳过NaN值， 默认为None

数学            NaN
英语      31.048349
计算机     40.139756
数英平均    25.861651
总成绩     85.663878
dtype: float64

In [284]:
df.std(numeric_only=True) # numeric_only是否只针对数值型

数学      48.589265
英语      31.048349
计算机     40.139756
数英平均    25.861651
总成绩     85.663878
dtype: float64

练习：
新建一个形状为10000*3的标准正态分布的DataFrame(np.random.randn)，去除掉所有满足以下情况的行：其中任一元素绝对值大于3倍标准差

In [289]:
(df > 100).all(axis=0)

数学      False
英语      False
计算机     False
数学名次     True
数英平均    False
总成绩      True
是否补考     True
名次       True
dtype: bool

In [291]:
ddd = DataFrame(np.random.randn(10000, 3),
                columns=('r', 'g', 'b'))
ddd.head()

,r,g,b
0,1.300813,-0.508417,-1.766171
1,-0.134917,-0.497554,-0.653554
2,0.231453,-0.953183,-0.843527
3,1.026438,0.101006,-0.499776
4,1.244582,0.387496,-0.348669


In [302]:
# 81行存在绝对值小于3倍的标准差
ddd[(np.abs(ddd) > ddd.std()*3).any(axis=1)]  

,r,g,b
75,-2.225594,3.088346,-1.401448
224,-3.525522,1.032002,0.229907
271,-3.697685,0.275799,0.629543
340,0.351129,3.678072,-1.045166
432,-0.525812,-1.223580,4.081183
599,-1.032930,-3.267971,0.417390
782,-0.773119,0.461525,3.310660
831,-1.199827,-3.356237,1.044527
1122,3.428278,-1.053222,-0.756194
1215,-0.024655,-3.221288,1.294860


In [306]:
# 获取要删除数据的索引
drop_index = ddd[(np.abs(ddd) > ddd.std()*3).any(axis=1)].index
drop_index

Int64Index([  75,  224,  271,  340,  432,  599,  782,  831, 1122, 1215, 1231,
            1305, 1344, 1555, 1598, 1633, 1686, 1742, 1893, 1962, 2185, 2414,
            2484, 2594, 2711, 2862, 3121, 3199, 3237, 3368, 3450, 3556, 3617,
            3751, 3796, 4006, 4167, 4428, 4439, 4464, 4541, 4633, 4642, 5019,
            5038, 5263, 5382, 5469, 5694, 5848, 6076, 6091, 6199, 6250, 6276,
            6384, 6530, 6684, 7055, 7350, 7451, 7490, 7687, 7755, 7758, 7824,
            8015, 8153, 8221, 8299, 8336, 8436, 8569, 8898, 9054, 9241, 9417,
            9578, 9626, 9713, 9725],
           dtype='int64')

In [307]:
# 注意：新版本drop(labels, axis=0, index, columns)
# ddd.drop(index=drop_index, inplace=True)
ddd.drop(drop_index)

,r,g,b
0,1.300813,-0.508417,-1.766171
1,-0.134917,-0.497554,-0.653554
2,0.231453,-0.953183,-0.843527
3,1.026438,0.101006,-0.499776
4,1.244582,0.387496,-0.348669
5,-0.538335,0.227585,0.545667
6,1.326753,-0.473597,-0.565738
7,-0.964959,0.698517,1.531014
8,0.419485,0.872058,1.011547
9,-1.252564,1.085063,1.798709


### 排列
- df.take(indices, axis)  按行索引，显示DF数据
- 借助于 np.random.permutation(n) 前n个数随机组合

In [310]:
df1.take([1, 0, 4]) # axis=0按行，按指定行索引的顺序显示数据

,数学,英语,计算机
B,119,11,120
A,34,79,35
E,47,109,35


In [311]:
df1.take(indices=[-1,1,0], axis=1) # axis=1按列排序

,计算机,英语,数学
A,35,79,34
B,120,11,119
C,36,81,44
D,98,97,46
E,35,109,47


In [323]:
np.random.permutation(5)  # 前n行或列的数据进行随机排序

array([1, 0, 4, 2, 3])

In [325]:
df1.take(np.random.permutation(5))  # 按行，随机排序

,数学,英语,计算机
D,46,97,98
B,119,11,120
E,47,109,35
A,34,79,35
C,44,81,36


In [340]:
np.random.seed(2)  # 随机种子
df1.take(np.random.permutation(3), axis=1) #按列， 随机排序 

,计算机,英语,数学
A,35,79,34
B,120,11,119
C,36,81,44
D,98,97,46
E,35,109,47


## 数据分类处理【重点】
- groupby([字段]).聚合函数()

In [343]:
persons = DataFrame({
    'id': ['101', '102', '103', '104', '105', '106'],
    'salary': [2000, 3000, 2500, 4000, 3800, 6000],
    'hire_date': [
        '2017',
        '2017',
        '2016',
        '2017',
        '2016',
        '2015'
    ],
    'sex':['男', '女', '男', '女', '女', '女']
    ,
    'perm': [.2, .25, .3, .2, .5, .35]
})
persons

,hire_date,id,perm,salary,sex
0,2017,101,0.20,2000,男
1,2017,102,0.25,3000,女
2,2016,103,0.30,2500,男
3,2017,104,0.20,4000,女
4,2016,105,0.50,3800,女
5,2015,106,0.35,6000,女


In [345]:
# 按性别分组，查看 2017年入职员工情况
persons.groupby('sex').groups

{'女': Int64Index([1, 3, 4, 5], dtype='int64'),
 '男': Int64Index([0, 2], dtype='int64')}

In [349]:
persons.groupby(['sex', 'hire_date']).groups

{('女', '2015'): Int64Index([5], dtype='int64'),
 ('女', '2016'): Int64Index([4], dtype='int64'),
 ('女', '2017'): Int64Index([1, 3], dtype='int64'),
 ('男', '2016'): Int64Index([2], dtype='int64'),
 ('男', '2017'): Int64Index([0], dtype='int64')}

In [354]:
persons.groupby(['hire_date', 'sex']).count().loc['2017']

,id,perm,salary
sex,,,
女,2,2,2
男,1,1,1


In [357]:
# 查看2016年入职员工的平均工资
persons.groupby(['hire_date'])['salary'].mean().loc['2016']

3150

In [359]:
# 查看各年度的工资支出
# 1. 计算每位员工的年薪
persons['salary_year'] = persons['salary'].mul(persons['perm']+1)*12

# 2. 分组，并对年薪进行汇总
persons.groupby(['hire_date'])['salary_year'].sum()

hire_date
2015     97200.0
2016    107400.0
2017    131400.0
Name: salary_year, dtype: float64

In [363]:
# 将列转成行索引
persons.set_index('id', inplace=True)

In [367]:
# 将索引转成列
# drop=True 删除索引（先将索引转成列，并且删除这一列），效果重新生索引
persons.reset_index()

,id,hire_date,perm,salary,sex,salary_year
0,101,2017,0.20,2000,男,28800.0
1,102,2017,0.25,3000,女,45000.0
2,103,2016,0.30,2500,男,39000.0
3,104,2017,0.20,4000,女,57600.0
4,105,2016,0.50,3800,女,68400.0
5,106,2015,0.35,6000,女,97200.0


In [368]:
persons

,hire_date,perm,salary,sex,salary_year
id,,,,,
101,2017,0.20,2000,男,28800.0
102,2017,0.25,3000,女,45000.0
103,2016,0.30,2500,男,39000.0
104,2017,0.20,4000,女,57600.0
105,2016,0.50,3800,女,68400.0
106,2015,0.35,6000,女,97200.0


练习：
假设菜市场张大妈在卖菜，有以下属性：
菜品(item)：萝卜，白菜，辣椒，冬瓜
颜色(color)：白，青，红
重量(weight)
价格(price)
要求以属性作为列索引，新建一个df = DataFrame
对df进行聚合操作，求出颜色为白色的价格总和
对df进行聚合操作，求出萝卜的所有重量(包括白萝卜，胡萝卜，青萝卜）以及平均价格

In [379]:
ccc = DataFrame({
    'item': ['萝卜','白菜','萝卜','辣椒', '白菜','萝卜','冬瓜', '辣椒'],
    'color':['白', '白', '红', '红', '青','红', '青', '青'],
    'weight':np.random.uniform(1.0, 5.0, size=8),
    'price': np.random.uniform(2.0, 100.0, size=8)
})
ccc = np.round(ccc,2)
ccc

,color,item,price,weight
0,白,萝卜,8.58,4.81
1,白,白菜,99.40,3.18
2,红,萝卜,97.12,1.33
3,红,辣椒,80.43,2.47
4,青,白菜,60.98,4.40
5,红,萝卜,76.97,2.63
6,青,冬瓜,18.58,1.11
7,青,辣椒,30.72,1.99


In [376]:
# 对df进行聚合操作，求出颜色为白色的价格总和
# 注意： 分组的列，变成了行索引
np.round(ccc.groupby('color')['price'].sum().loc['白'], 2)

140.17


对df进行聚合操作，求出萝卜的所有重量(包括白萝卜，胡萝卜，青萝卜）以及平均价格

In [381]:
g = ccc.groupby('item')
g['weight'].sum().loc['萝卜']

8.7699999999999996

In [382]:
g['price'].mean().loc['萝卜']

60.890000000000008

高级数据聚合
- transform()
- apply()

In [386]:
# 聚合后，调用add_prefix 添加前辍
ccc.groupby('color')['price', 'weight'].sum().add_prefix('t_')

,t_price,t_weight
color,,
白,107.98,7.99
红,254.52,6.43
青,110.28,7.50


In [392]:
# 使用transform() 转换函数
# 针对分组的特征，进行指定sum()聚合函数计算完成后，
# 按原有的索引生成计算后的数据
ccc.groupby('color')['price'].transform(sum).add_prefix('t_')

t_0    107.98
t_1    107.98
t_2    254.52
t_3    254.52
t_4    110.28
t_5    254.52
t_6    110.28
t_7    110.28
Name: price, dtype: float64

In [397]:
ccc3 = ccc.copy()
ccc3['t_weight'] = ccc.groupby('color')['weight'].transform(sum)
ccc3

,color,item,price,weight,t_weight
0,白,萝卜,8.58,4.81,7.99
1,白,白菜,99.40,3.18,7.99
2,红,萝卜,97.12,1.33,6.43
3,红,辣椒,80.43,2.47,6.43
4,青,白菜,60.98,4.40,7.50
5,红,萝卜,76.97,2.63,6.43
6,青,冬瓜,18.58,1.11,7.50
7,青,辣椒,30.72,1.99,7.50


In [398]:
ccc3['mean_price'] = ccc.groupby('color')['price'].transform(np.mean)
ccc3

,color,item,price,weight,t_weight,mean_price
0,白,萝卜,8.58,4.81,7.99,53.99
1,白,白菜,99.40,3.18,7.99,53.99
2,红,萝卜,97.12,1.33,6.43,84.84
3,红,辣椒,80.43,2.47,6.43,84.84
4,青,白菜,60.98,4.40,7.50,36.76
5,红,萝卜,76.97,2.63,6.43,84.84
6,青,冬瓜,18.58,1.11,7.50,36.76
7,青,辣椒,30.72,1.99,7.50,36.76


In [403]:
def trans_aggr(item):
    # item ->Series，代表每一个分组中某一特征的数值
#     print(item)
#     print(type(item))
    if item.name == 'price':
        return item.mean()  # 计算每一个分组的平均价
    
    return item.sum() # 计算每一个分组的总重量

In [405]:
# 一次性针对不同特征，进行不同的计算
# concat()合并
ccc4 = ccc.groupby('color')['price','weight'].transform(trans_aggr)
ccc4

,price,weight
0,53.99,7.99
1,53.99,7.99
2,84.84,6.43
3,84.84,6.43
4,36.76,7.50
5,84.84,6.43
6,36.76,7.50
7,36.76,7.50


In [408]:
ccc4.rename(columns={'price':'mean_price', 
                     'weight':'total_weight'},
           inplace=True)
ccc4

,mean_price,total_weight
0,53.99,7.99
1,53.99,7.99
2,84.84,6.43
3,84.84,6.43
4,36.76,7.50
5,84.84,6.43
6,36.76,7.50
7,36.76,7.50


In [409]:
# 将计算的结果以列级联方式，合并起来
pd.concat((ccc, ccc4), axis=1)

,color,item,price,weight,mean_price,total_weight
0,白,萝卜,8.58,4.81,53.99,7.99
1,白,白菜,99.40,3.18,53.99,7.99
2,红,萝卜,97.12,1.33,84.84,6.43
3,红,辣椒,80.43,2.47,84.84,6.43
4,青,白菜,60.98,4.40,36.76,7.50
5,红,萝卜,76.97,2.63,84.84,6.43
6,青,冬瓜,18.58,1.11,36.76,7.50
7,青,辣椒,30.72,1.99,36.76,7.50


In [422]:
def apply_aggr(item:DataFrame):
    # item -> DataFrame
    # print(item)
    # print(type(item))
    # display(item.sum())
    # 返回Seris->index['price', 'weight']
    # display(item['price'].mean(), item['weight'].sum())
    index = ['mean_price', 'total_weight']
    return Series([item['price'].mean(), 
                  item['weight'].sum()], index=index)

In [425]:
ccc.groupby('color')['price', 'weight'].apply(apply_aggr)

,mean_price,total_weight
color,,
白,53.99,7.99
红,84.84,6.43
青,36.76,7.50


### 总结
- 分组 groupby()
- 分组之后的操作
    - 通过 ['列名'] 选择某一列，当然选择多个列
    - 选择特征列之后，可以调用聚合函数
        - mean()  平均
        - max()   最大
        - min()   最小
        - sum()   求和
        - count() 统计数量
        
        另外， 聚合函数之后，还可以调用add_prefix('前辍')
    - 选择特征列之后，也可以调用apply()或transform()
        - 如果对不同特征列做不同聚合时，需要使用apply和transform
        - 相同之处， 都可以接收聚合函数（np.mean, np.sum等）和自定义函数
        - 不同之处
            - apply() 返回的数据不包含原有的索引，向自定义函数传的DataFrame
            - transform() 返回的数据包含原有的索引，向自定义函数传的Series
    